In [1]:
import librosa
import matplotlib.pyplot as plt
# Instead of %matplotlib inline we use %matplotlib notebook this time.
# This allows for more interactive examination of graphs, which will be useful
# as you will manually inspect the results curves.
%matplotlib inline
from scipy.signal import resample
from scipy.signal import find_peaks_cwt
from scipy.spatial import distance

from IPython.display import Audio
import cv2
import os
import datetime
import numpy as np
import os
import urllib
import IPython
import matplotlib.pyplot as plt
import librosa.display
import copy
from scipy.io import wavfile
from scipy.signal import butter, lfilter
import scipy.ndimage

In [2]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import HoverTool

In [43]:
# show the audio
IPython.display.display(IPython.display.Audio(AUDIO_PATH))

plt.figure(figsize=(180,2))
librosa.display.waveplot(librosa.amplitude_to_db(audio_signal))

threshold = 30
for item in librosa.amplitude_to_db(audio_signal):
    if item > threshold:
        print(item)
        
plt.figure(figsize=(50,2))  
sample_rate_set = sample_rate
librosa.display.waveplot(audio_signal, sample_rate_set) 
plt.show()


['audio/nba_start0_end5.mp3\n', 'audio/nba_start5_end10.mp3\n', 'audio/nba_start10_end15.mp3\n', 'audio/nba_start15_end20.mp3\n', 'audio/nba_start20_end25.mp3\n', 'audio/nba_start25_end27.mp3']
audio/nba_start0_end5.mp3


In [ ]:
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform

def hs_histograms_similarity(hs_histogram0, hs_histogram1):
  features_distance_metrics = {'intersection_area': lambda u, v: np.sum(np.min([u, v], 0)),
                              'cityblock': 'cityblock',
                              'seuclidean': 'seuclidean',
                              'correlation':'correlation',
                              'chebyshev': 'chebyshev',
                              'euclidean': 'euclidean', } # NB: this is a similarity metric.
  hist = np.matrix([hs_histogram0,hs_histogram1])
  similarity = 1 - pdist(hist, features_distance_metrics['correlation'])
  return similarity

def next_pow_2(x):
    """Smallest next power of two of a given value x."""
    return 1 << (x - 1).bit_length()

def compute_self_similarity(feature_vector_matrix, similarity_function = hs_histograms_similarity):
  # TODO: implement. 
  # For calling the similarity function, you can just use similarity_function(first_vector, second_vector).
  h, w = feature_vector_matrix.shape
  self_similarity = np.zeros((h,h))
  for i in range(h):
    for j in range(h):
      self_similarity[i,j] = similarity_function(feature_vector_matrix[i,:],feature_vector_matrix[j,:])
      
  return self_similarity

def scale_vector(v):
  return (v - np.min(v)) / (np.max(v) - np.min(v))

In [143]:
with open('audio/files.txt', 'r') as f:
    data = f.readlines()
print(len(data))
print(data[0][:-1])
print(data)

6
audio/nba_start0_end5.mp3
['audio/nba_start0_end5.mp3\n', 'audio/nba_start5_end10.mp3\n', 'audio/nba_start10_end15.mp3\n', 'audio/nba_start15_end20.mp3\n', 'audio/nba_start20_end25.mp3\n', 'audio/nba_start25_end27.mp3']


In [144]:
highlight2 = []
time = 0
for i in range(len(data)):

    if i != (len(data)-1):
        AUDIO_PATH = data[i][:-1] #os.path.join('/home/xin/Downloads/MMSR_Project/', 'nba_cut.mp3')
    else: 
        AUDIO_PATH = data[i]
        
    print(AUDIO_PATH)
    
    audio_signal, sample_rate = librosa.core.load(AUDIO_PATH)

    # Audio duration (in seconds).
    number_of_audio_samples = len(audio_signal)
    audio_duration = float(number_of_audio_samples / float(sample_rate))
    

    print('sample rate: %d' % sample_rate)
    print('number of samples: %d' % number_of_audio_samples)
    print('duration: %s' % datetime.timedelta(seconds=audio_duration))


    audio_frame_size = next_pow_2(int(sample_rate / 4.0))  # i.e., about 0.25 seconds.
    audio_hop_size = int(audio_frame_size / 2.0)  # i.e., 50% overlap.
    number_of_seconds = librosa.get_duration(audio_signal, sample_rate)

    # idx = slice(*librosa.time_to_frames([30, 35], sr=sample_rate))
    
    S_full, phase = librosa.magphase(librosa.stft(audio_signal))
    S_full = S_full
    S_filter = librosa.decompose.nn_filter(S_full,
                                           aggregate=np.median,
                                           metric='cosine',
                                           width=int(librosa.time_to_frames(1, sr=sample_rate)))

    S_filter = np.minimum(S_full, S_filter)


    margin_i, margin_v = 2, 10
    power = 2

    mask_i = librosa.util.softmask(S_filter,
                                   margin_i * (S_full - S_filter),
                                   power=power)

    mask_v = librosa.util.softmask(S_full - S_filter,
                                   margin_v * S_filter,
                                   power=power)

    # Once we have the masks, simply multiply them with the input spectrum
    # to separate the components

    S_foreground = mask_v * S_full
    S_background = mask_i * S_full

    S_full_recover = S_full * phase;
    S_full_istft = librosa.istft(S_full_recover);


    S_background_recover = S_background * phase;
    S_background_istft = librosa.istft(S_background_recover);

    S_foreground_recover = S_foreground * phase;
    S_foreground_istft = librosa.istft(S_foreground_recover);

    melspec = librosa.feature.melspectrogram(S_foreground_istft, sample_rate,  n_fft=1024, hop_length=512, n_mels=128)  
    # convert to log scale  
    logmelspec = librosa.power_to_db(melspec)  

    mat = []
    for i in range(int(logmelspec.shape[1]*512/sample_rate)):
        sec = logmelspec[:,int(i * sample_rate/512 ): 
                         int((i + 1) * sample_rate/512)]
        sec = scale_vector(sec)
        mat.append(np.mean(sec, axis =1))

    ##### SET THRESHOLD
    threshold = 0.5 #-35
    ran = int(logmelspec.shape[1]*512/sample_rate)
    print(ran)

    for i in range(ran):
        m = np.mean(mat[i][20:100])
        st = np.std(mat[i][20:100])
        if  m > threshold:
            highlight2.append(i+time)
            print(i//60 , i % 60)
            print(m)
            print(st)
            print('-------------------------------')
            
    time = time + audio_duration

audio/nba_start0_end5.mp3
sample rate: 22050
number of samples: 6615936
duration: 0:05:00.042449
300
0 7
0.5062770098447307
0.05322931978012291
-------------------------------
0 41
0.5318979569483459
0.0880126702285601
-------------------------------
0 42
0.5612946643518537
0.08697036474305968
-------------------------------
audio/nba_start5_end10.mp3
sample rate: 22050
number of samples: 6615936
duration: 0:05:00.042449
300
0 55
0.5719148572097101
0.05667769615768043
-------------------------------
0 58
0.5168645431545748
0.07030198823407831
-------------------------------
0 59
0.5292367379839247
0.10262768373610161
-------------------------------
1 0
0.5150119353534985
0.09699327046172872
-------------------------------
1 1
0.5330912803825806
0.10080938033895262
-------------------------------
1 25
0.5337400076788661
0.10932356434922348
-------------------------------
2 13
0.5158946522952123
0.04787198139548301
-------------------------------
2 15
0.5572130078132137
0.074633663271674

In [145]:
highlight_2 = highlight2.copy()
prev_time = 0
for time in highlight2:
    if time - prev_time < 5:
        
        highlight_2.remove(time)
    prev_time = time    
        

print(len(highlight2),len(highlight_2))
#print(highlight)
print(highlight_2)

28 15
[7, 41, 355.0424489795918, 385.0424489795918, 433.0424489795918, 513.0424489795919, 753.0848979591836, 810.0848979591836, 815.0848979591836, 961.1273469387754, 1104.1273469387754, 1131.1273469387754, 1306.1697959183673, 1432.1697959183673, 1502.2122448979592]


In [146]:
highlight_1 = highlight_2
file_list = []
name_ = 'file '
with open('files.txt','w') as f:
    for idx,time in enumerate(highlight_1):
        start = str(time-10)
        end = str(time+3)
        if idx != len(highlight_1):
            file_name = name_+"'video2/cut"+str(idx)+".mp4'\n"
        else:
            file_name = name_+"'video2/cut"+str(idx)+".mp4'\n"
        f.write(str(file_name))
        

In [147]:
#!ffmpeg -ss 00:01:00 -i nba.mp4 -to 00:02:00 -c copy cut.mp4
import ffmpeg
import os
file_list = []
name_ = ' video2/cut'
for idx,time in enumerate(highlight_1):
    start = str(time-11)
    if idx == 0:
        if int(start) < 0:
            start = str(0)
    end = str(time+4)
    
    file_name = name_+str(idx)+'.mp4'
    command = "ffmpeg -ss "+start+" -to " + end + " -accurate_seek -i nba.mp4 -codec copy -avoid_negative_ts 1 " + file_name
    #print(command)
    os.system(command)

command_concat = "ffmpeg -f concat -i files.txt -map 0 -fflags +genpts -c copy video2/movie3.mp4"

os.system(command_concat)

0

# Evaluation 

In [148]:
label1 = [9,257,313,360,437,629,674,714,755,787,814,987,1072,1133,1181,1236,1273,1295,1310,1364,1382,1416,1440]
label2 = [24,65,86,94,121,147,242,271,416,448,500,629,656,673,810,821,842,956,1023,1084,1192,1208,1251,1268,1320,1340,1468,1513,1536,1567,1590,1602]
print(len(label1))
print(len(label2))

highlight = [int(i) for i in highlight_1]
print("highlight -",str(len(highlight)))
print(highlight)

23
32
highlight - 15
[7, 41, 355, 385, 433, 513, 753, 810, 815, 961, 1104, 1131, 1306, 1432, 1502]


In [151]:
label_list = [list(range(i-5,i+5)) for i in label1]
hl_list = [list(range(i-5,i+5)) for i in highlight]
#highlight_1  

def range2list(l):
    a = []
    for i in l:
        a = a+i
    #print(a)
    return a

label_range = range2list(label_list)
hl_range = range2list(hl_list)
print(hl_range)

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 508, 509, 510, 511, 512, 513, 514, 515, 516, 517, 748, 749, 750, 751, 752, 753, 754, 755, 756, 757, 805, 806, 807, 808, 809, 810, 811, 812, 813, 814, 810, 811, 812, 813, 814, 815, 816, 817, 818, 819, 956, 957, 958, 959, 960, 961, 962, 963, 964, 965, 1099, 1100, 1101, 1102, 1103, 1104, 1105, 1106, 1107, 1108, 1126, 1127, 1128, 1129, 1130, 1131, 1132, 1133, 1134, 1135, 1301, 1302, 1303, 1304, 1305, 1306, 1307, 1308, 1309, 1310, 1427, 1428, 1429, 1430, 1431, 1432, 1433, 1434, 1435, 1436, 1497, 1498, 1499, 1500, 1501, 1502, 1503, 1504, 1505, 1506]


In [152]:
tr= [i in hl_range for i in label1]
print(tr)
print(len(tr))

t_p = sum(tr)
print("True Positive: ", str(t_p))
f_n = len(label) - t_p
print("False Negative: ", str(f_n))

recall = t_p / (t_p+f_n)
print("Recall: ", str(recall))

[True, False, False, False, True, False, False, False, True, False, True, False, False, True, False, False, False, False, True, False, False, False, False]
23
True Positive:  6
False Negative:  17
Recall:  0.2608695652173913


In [123]:
print(7 in hl_range)

True
